## Pozyskanie danych
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne pakiety

In [ ]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt

## <center>Część 2</center>
### 2.5

In [ ]:
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"] = "projekt1-415610-f19ebf54e064.json"  # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()

### Potrzebna funkcja

In [ ]:
def return_df(q):
    query_job = client.query(q)
    query_result = query_job.result()
    return query_result.to_dataframe()

### 2.6

In [ ]:
query = 'select * from `bigquery-public-data.covid19_open_data.covid19_open_data` limit 10'
df = return_df(query)

### Informacje o zbiorze

In [ ]:
df.info()

## <center>Część 3</center>

### 3.1. Sprawdź, ile jest zapisanych wierszy z danymi.

In [ ]:
df_3_1 = return_df('SELECT COUNT(*) FROM `bigquery-public-data.covid19_open_data.covid19_open_data`')
df_3_1

### 3.2. Sprawdź, ile krajów jest uwzględnionych w danych.

In [ ]:
df_3_2 = return_df('SELECT COUNT(DISTINCT country_name) FROM `bigquery-public-data.covid19_open_data.covid19_open_data`')
df_3_2

### 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.

In [ ]:
df_3_3 = return_df('SELECT column_name, data_type FROM `bigquery-public-data`.covid19_open_data.INFORMATION_SCHEMA.COLUMNS WHERE table_name = \'covid19_open_data\''
)
df_3_3

### 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.

In [ ]:
df_3_4 = return_df('SELECT column_name, data_type FROM `bigquery-public-data`.covid19_open_data.INFORMATION_SCHEMA.COLUMNS WHERE table_name = \'covid19_open_data\' AND data_type = \'INT64\' OR data_type = \'FLOAT64\''
)
df_3_4

### 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.

In [ ]:
queries_3_5 = [
    'SELECT MIN(date) AS min_date, MAX(date) AS max_date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`',
    'SELECT MIN(date) AS min_date_confirmed, MAX(date) AS max_date_confirmed FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_confirmed IS NOT NULL',
    'SELECT MIN(date) AS min_date_deceased, MAX(date) AS max_date_deceased FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_deceased IS NOT NULL',
    'SELECT MIN(date) AS min_date_vaccinated, MAX(date) AS max_date_vaccinated FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_persons_vaccinated IS NOT NULL'
]

df_3_5 = pd.DataFrame()

for query in queries_3_5:
    df_3_5 = pd.concat([df_3_5, return_df(query)], axis=1)
df_3_5

### 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń.

## <center>Część 4</center>

In [ ]:
# count_to_del = pd.DataFrame(list(zip([df[el].notna().sum() for el in df.columns], df.columns)), columns=['count', 'names'])
# stats = count_to_del['count'].describe().drop(['count', 'min', 'max'])
# stats

### Od ilu pustych wierszy usunąć kolumnę?

In [ ]:
# colors = ['red', 'green', 'pink', 'yellow', 'orange']
# fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
# count_to_del['count'].plot(kind='line', color='blue', ax=ax[0])
# for i, el in enumerate(stats.index):
#     ax[0].axhline(stats.iloc[i], color=colors[i], linestyle='solid', label=el)
# ax[0].legend(loc='upper center')
# count_to_del['count'].plot(kind='box', ax=ax[1])

### Usunięcie kolumn zawierających za dużo wartości NaN

In [ ]:
# count_to_del = count_to_del[count_to_del['count'] > 2 * count_to_del['count'].std()]
# df = df[count_to_del['names']]

### Usunięcie wierszy zawierających wartości NaN

In [ ]:
# df.dropna(inplace=True)
# df.drop(['place_id', 'wikidata_id', 'datacommons_id', 'country_code', 'iso_3166_1_alpha_2', 'new_hospitalized_patients_age_3', 'new_hospitalized_patients_male'], axis=1, inplace=True)

### Usunięcie duplikatów

In [ ]:
# df.drop_duplicates(inplace=True, ignore_index=True)

### Czyszczenie danych

In [ ]:
# count_to_del

### Gotowe dane

In [ ]:
df.info()

In [ ]:
df.head()

### 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [ ]:
query_4_1 = 'SELECT DISTINCT country_name, population, population_male, population_female, gdp_usd, gdp_per_capita_usd, latitude, longitude, area_sq_km, infant_mortality_rate, health_expenditure_usd, human_capital_index, life_expectancy, population_age_00_09 AS age_bin_0, population_age_10_19 AS age_bin_1, population_age_20_29 AS age_bin_2, population_age_30_39 AS age_bin_3, population_age_40_49 AS age_bin_4, population_age_50_59 AS age_bin_5, population_age_60_69 AS age_bin_6, population_age_70_79 AS age_bin_7, population_age_80_and_older AS age_bin_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 ORDER BY country_name'

df_4_1 = return_df(query_4_1)

In [ ]:
df_4_1.info()

### Uzupełnienie brakujących danych o kraju na podstawie jego <i>sąsiadów</i>

In [ ]:
from haversine import haversine

def save_to_csv(dataframe, filename):
    if 'czesc4' not in os.listdir(os.getcwd()):
        os.mkdir('czesc4')
    os.chdir('czesc4')
    dataframe.to_csv(filename + '.csv') # Zapis do csv
    os.chdir('..')
    
def fill_based_on_neigh(index, num=10):
    data = df_4_1.loc[index].copy()
    df_4_1['distance'] = df_4_1.apply(lambda x: haversine((data['latitude'], data['longitude']), (x['latitude'], x['longitude'])), axis=1)
    neigh = df_4_1.sort_values(by=['distance'])
    neigh = neigh.iloc[0:num]
    for col in columns_1.difference(leave_1):
        if df_4_1[col].dtype == np.float64:
            df_4_1.loc[index, col] = neigh[col].mean()
        else:
            df_4_1.loc[index, col] = int(neigh[col].mean())
            
def fill_4_1():
    df_4_1['distance'] = 0.0
    to_float_cols = columns_1.difference(leave_1)
    for tfc in to_float_cols:
         df_4_1[tfc] = df_4_1[tfc].astype(np.float64)
    df_4_1['population'].fillna(df_4_1['population_male'] + df_4_1['population_female'], inplace=True)
    df_4_1['population_male'].fillna(df_4_1['population'] // 2, inplace = True)
    df_4_1['population_female'].fillna(df_4_1['population'] // 2, inplace = True)
    for index, row in df_4_1.iterrows():
        if df_4_1.loc[index].isna().sum() > 0:
            fill_based_on_neigh(index=index)
    df_4_1.drop('distance', axis=1, inplace=True)
    
    split = [9, 9, 8, 7, 7, 8, 9, 9]
    for i in range(len(split)):
        df_4_1[age_groups[i]].fillna(df_4_1['population'] // split[i], inplace=True)
    df_4_1['age_bin_8'].fillna(df_4_1['population'] - df_4_1[age_groups].sum(axis=1), inplace=True)
    
    for rn in columns_1[df_4_1.isna().any()]:
        df_4_1[rn].fillna(df_4_1[rn].mode()[0], inplace=True)


columns_1 = df_4_1.columns
age_groups = [el for el in columns_1 if 'age_bin_' in el]
leave_1 = ['population', 'population_male', 'population_female', 'latitude', 'longitude', 'country_name'] + age_groups

fill_4_1()
save_to_csv(df_4_1, 'df_4_1')

### Informacje o krajach

In [ ]:
df_4_1.info()

In [ ]:
df_4_1.head()

### 4.2 Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [ ]:
query_4_2 = 'SELECT country_name, date, SUM(new_confirmed) AS daily_confirmed, SUM(new_confirmed_male) AS daily_confirmed_male, SUM(new_confirmed_female) AS daily_confirmed_female, SUM(new_confirmed_age_0) AS age_bin_0, SUM(new_confirmed_age_1) AS age_bin_1, SUM(new_confirmed_age_2) AS age_bin_2, SUM(new_confirmed_age_3) AS age_bin_3, SUM(new_confirmed_age_4) AS age_bin_4, SUM(new_confirmed_age_5) AS age_bin_5, SUM(new_confirmed_age_6) AS age_bin_6, SUM(new_confirmed_age_7) AS age_bin_7, SUM(new_confirmed_age_8) + SUM(new_confirmed_age_9) AS age_bin_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` GROUP BY country_name, date ORDER BY country_name, date'
 
df_4_2 = return_df(query_4_2)

In [ ]:
df_4_2.info()

### Uzupełnienie danych dotyczących zachorowań na COVID-19

In [ ]:
def fill_4_2():
    df_4_2.select_dtypes(include=np.number).clip(lower=0, inplace=True)
    
    df_4_2['daily_confirmed'].fillna(0, inplace=True)
    
    mask = df_4_2['daily_confirmed_male'].isna() | df_4_2['daily_confirmed_female'].isna()
    rand = np.random.randint(1, 10, size=len(mask))
    df_4_2.loc[mask, 'daily_confirmed_male'] = df_4_2['daily_confirmed'].where(mask, df_4_2['daily_confirmed'] // rand)
    df_4_2.loc[mask, 'daily_confirmed_female'] = df_4_2['daily_confirmed'].where(mask, df_4_2['daily_confirmed'] - df_4_2['daily_confirmed_male'])
    
    mask_update = (df_4_2['daily_confirmed'] == 0) & (df_4_2['daily_confirmed_male'] != 0) & (df_4_2['daily_confirmed_female'] != 0)
    df_4_2.loc[mask_update, 'daily_confirmed'] = df_4_2['daily_confirmed_male'] + df_4_2['daily_confirmed_female']

ratio = pd.DataFrame({'country_name': df_4_1['country_name']})
for age_group in age_groups:
    ratio[age_group] = df_4_1[age_group] / df_4_1['population']

fill_4_2()
save_to_csv(df_4_2, 'df_4_2')

### Dzienne statystyki zachorowań na COVID-19 dla każdego kraju

In [ ]:
df_4_2.info()

In [ ]:
df_4_2.head()

### 4.3 Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

In [ ]:
query_4_3 = 'SELECT country_name, date, SUM(new_deceased) AS daily_deceased, SUM(new_deceased_male) AS daily_deceased_male, SUM(new_deceased_female) AS daily_deceased_female, SUM(new_deceased_age_0) AS age_bin_0, SUM(new_deceased_age_1) AS age_bin_1, SUM(new_deceased_age_2) AS age_bin_2, SUM(new_deceased_age_3) AS age_bin_3, SUM(new_deceased_age_4) AS age_bin_4, SUM(new_deceased_age_5) AS age_bin_5, SUM(new_deceased_age_6) AS age_bin_6, SUM(new_deceased_age_7) AS age_bin_7, SUM(new_deceased_age_8) + SUM(new_deceased_age_9) AS age_bin_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` GROUP BY country_name, date ORDER BY country_name, date'

df_4_3 = return_df(query_4_3)

In [ ]:
df_4_3.info()

### Uzupełnienie danych dotyczących śmiertelności COVID-19

### 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

### 4.5. Zdefiniuj własny dodatkowy przypadek.